In [2]:
import os
from time import time
import re
import shutil
import glob
import warnings

BASE_DIR = "/hps_demo"
embedding_folder  = os.path.join(BASE_DIR, "embedding")
wdl_embedding_repo= os.path.join(embedding_folder, "hps_infer")
wdl_version =os.path.join(wdl_embedding_repo, "1")

if os.path.isdir(embedding_folder):
    shutil.rmtree(embedding_folder)
os.makedirs(embedding_folder)

if os.path.isdir(wdl_embedding_repo):
    shutil.rmtree(wdl_embedding_repo)
os.makedirs(wdl_embedding_repo)

if os.path.isdir(wdl_version):
    shutil.rmtree(wdl_version)
os.makedirs(wdl_version)

In [3]:
!tree -l $BASE_DIR

/hps_demo
└── embedding
    └── hps_infer
        └── 1

3 directories, 0 files


In [4]:
!tree -l hps_model

hps_model
├── hps_train.json
├── hps_train0_opt_sparse_1000.model
├── hps_train0_sparse_1000.model
│   ├── emb_vector
│   └── key
├── hps_train1_opt_sparse_1000.model
├── hps_train1_sparse_1000.model
│   ├── emb_vector
│   └── key
├── hps_train_dense_1000.model
├── hps_train_label_1000
├── hps_train_opt_dense_1000.model
├── hps_train_pred_1000
└── infer_test.csv

2 directories, 12 files


In [42]:
!cp -r ./hps_model/hps_train0_sparse_1000.model /hps_demo/embedding/hps_infer/1
!cp -r ./hps_model/hps_train1_sparse_1000.model /hps_demo/embedding/hps_infer/1
!cp ./hps_model/hps_train_dense_1000.model /hps_demo/embedding/hps_infer/1
!cp ./hps_model/hps_train.json /hps_demo/embedding/hps_infer/1
!tree -l /hps_demo

/hps_demo
└── embedding
    ├── hps.json
    └── hps_infer
        ├── 1
        │   ├── hps_train.json
        │   ├── hps_train0_sparse_1000.model
        │   │   ├── emb_vector
        │   │   └── key
        │   ├── hps_train1_sparse_1000.model
        │   │   ├── emb_vector
        │   │   └── key
        │   └── hps_train_dense_1000.model
        ├── config.pbtxt
        └── infer_test.csv

5 directories, 9 files


In [ ]:
# TODO: modify instance_group, config with 3 models

In [1]:
%%writefile /hps_demo/embedding/hps_infer/config.pbtxt
name: "hps_train"
backend: "hps"
max_batch_size:1024,
input [
   {
    name: "DES"
    data_type: TYPE_FP32
    dims: [ -1 ]
  },
  {
    name: "CATCOLUMN"
    data_type: TYPE_INT64
    dims: [ -1 ]
  }
]
output [
  {
    name: "OUTPUT0"
    data_type: TYPE_FP32
    dims: [ -1 ]
  }
]
version_policy: {
        specific:{versions: 1}
},
instance_group [
  {
    count: 1
    kind : KIND_GPU
    gpus:[0]
  }
]
parameters [
  {
  key: "config"
  value: { string_value: "/hps_demo/embedding/hps_infer/1/hps_train.json" }
  },
  {
  key: "gpucache"
  value: { string_value: "true" }
  },
  {
  key: "hit_rate_threshold"
  value: { string_value: "0.8" }
  },
  {
  key: "gpucacheper"
  value: { string_value: "0.5" }
  },
  {
  key: "label_dim"
  value: { string_value: "1" }
  },
  {
  key: "slots"
  value: { string_value: "28" }
  }
]

Overwriting /hps_demo/embedding/hps_infer/config.pbtxt


In [2]:
%%writefile /hps_demo/embedding/hps.json
{
    "supportlonglong": true,
    # "volatile_db": {
    #     "type": "hash_map",
    #     "user_name": "default",
    #     "num_partitions": 8,
    #     "max_get_batch_size": 1024,
    #     "max_set_batch_size": 1024,
    #     "overflow_policy": "evict_oldest",
    #     "overflow_margin": 1024,
    #     "overflow_resolution_target": 0.8,
    #     "initial_cache_rate": 1.0
    # },
    # "persistent_db": {
    #     "type": "disabled"
    # },
    "models": [{
        "model": "hps_train",
        "sparse_files": ["/hps_demo/embedding/hps_infer/1/hps_train0_sparse_1000.model", "/hps_demo/embedding/hps_infer/1/hps_train1_sparse_1000.model"],
        "dense_file":"/hps_demo/embedding/hps_infer/1/hps_train_dense_1000.model",
        "network_file":"/hps_demo/embedding/hps_infer/1/hps_train.json",
        "num_of_worker_buffer_in_pool": 4,
        "embedding_table_names":["embedding_table1","embedding_table2"],
        "embedding_vecsize_per_table":[1,16],
        "maxnum_catfeature_query_per_table_per_sample":[2,26],
        "default_value_for_each_table":[0.0,0.0],
        "deployed_device_list":[0],
        "max_batch_size":1024,
        "cache_refresh_percentage_per_iteration":0.2,
        "hit_rate_threshold":0.8,
        "gpucacheper":0.5,
        "gpucache":true,
        "maxnum_des_feature_per_sample": 13,
        "slot_num":28
        }
    ]
}

Overwriting /hps_demo/embedding/hps.json


In [3]:
!tree -l /hps_demo

/hps_demo
└── embedding
    ├── hps.json
    └── hps_infer
        ├── 1
        │   ├── hps_train.json
        │   ├── hps_train0_sparse_1000.model
        │   │   ├── emb_vector
        │   │   └── key
        │   ├── hps_train1_sparse_1000.model
        │   │   ├── emb_vector
        │   │   └── key
        │   └── hps_train_dense_1000.model
        ├── config.pbtxt
        └── infer_test.csv

5 directories, 9 files


In [ ]:
!mkdir /usr/local/hugectr/backends/hps

In [52]:
!tree -l /usr/local/hugectr/backends

/usr/local/hugectr/backends
├── hps
│   └── libtriton_hps.so
└── hugectr
    └── libtriton_hugectr.so

2 directories, 2 files


In [53]:
# !cp /workspace/merlin/hugectr_inference_backend/build/libtriton_hugectr.so /usr/local/hugectr/backends/hugectr/

In [ ]:
!tritonserver --model-repository=/hps_demo/embedding/ --load-model=hps_train \
    --model-control-mode=explicit \
    --backend-directory=/usr/local/hugectr/backends \
    --backend-config=hps,ps=/hps_demor/embedding/hps.json
# !tritonserver --model-repository=/hps_demo/embedding/ --load-model=hps_train --model-control-mode=explicit --backend-directory=/usr/local/hugectr/backends --backend-config=hps,ps=/hps_demo/embedding/hps.json

In [13]:
!ls -l ./data_parquet/val

total 8196
-rw-r--r-- 1 root root     747 Jun 27 07:47 _metadata.json
-rw-r--r-- 1 root root 2094641 Jun 27 07:46 gen_0.parquet
-rw-r--r-- 1 root root 2093496 Jun 27 07:46 gen_1.parquet
-rw-r--r-- 1 root root 2094009 Jun 27 07:46 gen_2.parquet
-rw-r--r-- 1 root root 2093936 Jun 27 07:46 gen_3.parquet


In [14]:
import pandas as pd
df = pd.read_parquet("./data_parquet/val/gen_0.parquet")
df.head()

,_col0,_col1,_col2,_col3,_col4,_col5,_col6,_col7,_col8,_col9,_col10,_col11,_col12,_col13,_col14
0,0.369659,0.562382,0.268092,0.270976,0.425515,0.436015,0.794521,0.969748,0.118850,0.317950,0.050502,1,434,1026,13
1,0.105705,0.476559,0.835602,0.103531,0.256015,0.835396,0.476131,0.923220,0.899870,0.008765,0.691802,0,0,23,9
2,0.818356,0.255720,0.896250,0.023801,0.751817,0.846724,0.261466,0.645097,0.173824,0.348452,0.533557,2,11,184,7
3,0.201031,0.303372,0.502298,0.366995,0.754150,0.270130,0.811643,0.322071,0.037592,0.338294,0.373525,2,0,10,0
4,0.923416,0.532531,0.791524,0.313665,0.763071,0.649208,0.176048,0.956767,0.219100,0.629382,0.667392,50,9,1,46


In [15]:
CATEGORICAL_COLUMNS=["C" + str(x) for x in range(1, 5)]
CONTINUOUS_COLUMNS=["I" + str(x) for x in range(1, 11)]
LABEL_COLUMNS = ['label']
cols = LABEL_COLUMNS + CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS
cols

['label',
 'I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'C1',
 'C2',
 'C3',
 'C4']

In [16]:
df.set_axis(cols, axis=1,inplace=True)
df.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,C1,C2,C3,C4
0,0.369659,0.562382,0.268092,0.270976,0.425515,0.436015,0.794521,0.969748,0.118850,0.317950,0.050502,1,434,1026,13
1,0.105705,0.476559,0.835602,0.103531,0.256015,0.835396,0.476131,0.923220,0.899870,0.008765,0.691802,0,0,23,9
2,0.818356,0.255720,0.896250,0.023801,0.751817,0.846724,0.261466,0.645097,0.173824,0.348452,0.533557,2,11,184,7
3,0.201031,0.303372,0.502298,0.366995,0.754150,0.270130,0.811643,0.322071,0.037592,0.338294,0.373525,2,0,10,0
4,0.923416,0.532531,0.791524,0.313665,0.763071,0.649208,0.176048,0.956767,0.219100,0.629382,0.667392,50,9,1,46


In [17]:
df.to_csv('./hps_model/infer_test.csv', sep=',', index=False,header=True)

In [31]:
!cp ./hps_model/infer_test.csv /hps_demo/embedding/hps_infer

In [7]:
!curl -v localhost:8000/v2/health/ready

*   Trying 127.0.0.1:8000...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8000 (#0)
> GET /v2/health/ready HTTP/1.1
> Host: localhost:8000
> User-Agent: curl/7.68.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Content-Length: 0
< Content-Type: text/plain
< 
* Connection #0 to host localhost left intact


In [8]:
import tritonhttpclient

try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))
triton_client.is_server_live()

client created.
GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

In [9]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '22'}>
bytearray(b'[{"name":"hps_infer"}]')


[{'name': 'hps_infer'}]

In [6]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '46'}>
bytearray(b'[{"name":"wdl","version":"1","state":"READY"}]')


[{'name': 'wdl', 'version': '1', 'state': 'READY'}]

In [41]:
model_name = "hps_infer"
triton_client.load_model(model_name=model_name)

POST /v2/repository/models/wdl/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'wdl'


In [34]:
# %%writefile demo2predict.py
from tritonclient.utils import *
import tritonclient.http  as httpclient
import numpy as np
import pandas as pd
import sys

model_name = 'hps_infer'
CATEGORICAL_COLUMNS=["C" + str(x) for x in range(1, 5)]
CONTINUOUS_COLUMNS=["I" + str(x) for x in range(1, 11)]
LABEL_COLUMNS = ['label']
emb_size_array = [10000, 10000, 10000, 10000]
shift = np.insert(np.cumsum(emb_size_array), 0, 0)[:-1]
test_df=pd.read_csv("./hps_model/infer_test.csv",sep=',')

with httpclient.InferenceServerClient("localhost:8000") as client:
    dense_features = np.array([list(test_df[CONTINUOUS_COLUMNS].values.flatten())],dtype='float32')
    embedding_columns = np.array([list((test_df[CATEGORICAL_COLUMNS]+shift).values.flatten())],dtype='int64')
    # row_ptrs = np.array([list(range(0,21))+list(range(0,261))],dtype='int32')
    
    inputs = [
        httpclient.InferInput("KEYS", dense_features.shape,
                              np_to_triton_dtype(dense_features.dtype)),
        httpclient.InferInput("NUMKEYS", embedding_columns.shape,
                              np_to_triton_dtype(embedding_columns.dtype)),
        # httpclient.InferInput("ROWINDEX", row_ptrs.shape,
        #                       np_to_triton_dtype(row_ptrs.dtype)),
    ]

    inputs[0].set_data_from_numpy(dense_features)
    inputs[1].set_data_from_numpy(embedding_columns)
    # inputs[2].set_data_from_numpy(row_ptrs)
    outputs = [
        httpclient.InferRequestedOutput("OUTPUT0")
    ]

    response = client.infer(model_name,
                            inputs,
                            request_id=str(1),
                            outputs=outputs)

    result = response.get_response()
    print(result)
    print("Prediction Result:")
    print(response.as_numpy("OUTPUT0"))

InferenceServerException: Request for unknown model: 'hps_infer' is not found

In [28]:
!python3 ./demo2predict.py

Traceback (most recent call last):
  File "./demo2predict.py", line 36, in <module>
    response = client.infer(model_name,
  File "/usr/local/lib/python3.8/dist-packages/tritonclient/http/__init__.py", line 1372, in infer
    _raise_if_error(response)
  File "/usr/local/lib/python3.8/dist-packages/tritonclient/http/__init__.py", line 64, in _raise_if_error
    raise error
tritonclient.utils.InferenceServerException: Request for unknown model: 'hps_train' is not found
